In [133]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [134]:
# TWES臺灣證券交易所_每日收盤行情(ETF)_HTML版
date = input("請輸入西元年月日，例：20240522")
# https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date=20241009&type=0099P&response=html
url = "https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date=" + date + "&type=0099P&response=html"
print(url)

https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date=20241016&type=0099P&response=html


In [135]:
resp = requests.get(url)
resp

<Response [200]>

In [136]:
soup = BeautifulSoup(resp.text, "lxml")
soup

<!DOCTYPE html>
<html lang="zh">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title> 報表 - TWSE 臺灣證券交易所 </title>
<style>
    table { border-collapse: collapse; margin: 1em 0; }
    table, th, td { border: 1px solid lightgray; }
    table thead th, table thead td { text-align: center; background-color: #eee; }
    table thead div { font-size: 1.5em; padding: 10px; }
    table tbody tr:nth-child(even){ background-color:#f8f8f8; }
    td { padding: 4px; }
    td>p { margin: 0; text-align: center; }
    div.notes { margin: 2em 0 4em 0; line-height: 1.5em; }
    table tbody td { text-align: right; }
    table tbody td:first-child { text-align: left; }
    table tbody td:first-child + td { text-align: right; }
    </style>
</head>
<body>
<div>
<table>
<thead>
<tr>
<th colspan="16">
<div>113年10月16日 每日收盤行情(ETF)</div>
</th>
</tr>
<tr>
<th>證券代號</th>
<th>證券名稱</th>
<th>成交股數</th>
<th>成交筆數</th>
<th>成交金額</th>
<th>開盤

In [137]:
i = soup.find("tbody")

j = soup.find("tbody").text

k = soup.find("tbody").text.strip()

print(i), type(i), type(j), type(k)

<tbody>
<tr align="center" style="font-size:14px;">
<td>0050</td>
<td>元大台灣50</td>
<td>18,604,770</td>
<td>22,549</td>
<td>3,594,464,975</td>
<td>192.45</td>
<td>195.30</td>
<td>191.65</td>
<td>193.00</td>
<td><p style="color:green">-</p></td>
<td>3.40</td>
<td>193.00</td>
<td>113</td>
<td>193.05</td>
<td>41</td>
<td>0.00</td>
</tr>
<tr align="center" style="font-size:14px;">
<td>0051</td>
<td>元大中型100</td>
<td>115,782</td>
<td>353</td>
<td>9,375,296</td>
<td>81.00</td>
<td>81.30</td>
<td>80.80</td>
<td>80.80</td>
<td><p style="color:green">-</p></td>
<td>0.60</td>
<td>80.80</td>
<td>3</td>
<td>80.90</td>
<td>2</td>
<td>0.00</td>
</tr>
<tr align="center" style="font-size:14px;">
<td>0052</td>
<td>富邦科技</td>
<td>1,391,669</td>
<td>1,974</td>
<td>266,184,740</td>
<td>189.90</td>
<td>193.70</td>
<td>189.45</td>
<td>191.10</td>
<td><p style="color:green">-</p></td>
<td>3.05</td>
<td>191.05</td>
<td>1</td>
<td>191.10</td>
<td>13</td>
<td>0.00</td>
</tr>
<tr align="center" style="font-size:14px

(None, bs4.element.Tag, str, str)

In [138]:
if soup.find("tbody").text.strip() == "":
    print("休市日")

In [139]:
title = soup.find("thead").find("div").text
title

'113年10月16日 每日收盤行情(ETF)'

In [140]:
ths = [ th.text for th in soup.find("thead").find_all("th")[1:] ]
ths

['證券代號',
 '證券名稱',
 '成交股數',
 '成交筆數',
 '成交金額',
 '開盤價',
 '最高價',
 '最低價',
 '收盤價',
 '漲跌(+/-)',
 '漲跌價差',
 '最後揭示買價',
 '最後揭示買量',
 '最後揭示賣價',
 '最後揭示賣量',
 '本益比']

In [141]:
tds = [ [ td.text for td in tr.find_all("td") ] for tr in soup.find("tbody").find_all("tr") ]
tds

[['0050',
  '元大台灣50',
  '18,604,770',
  '22,549',
  '3,594,464,975',
  '192.45',
  '195.30',
  '191.65',
  '193.00',
  '-',
  '3.40',
  '193.00',
  '113',
  '193.05',
  '41',
  '0.00'],
 ['0051',
  '元大中型100',
  '115,782',
  '353',
  '9,375,296',
  '81.00',
  '81.30',
  '80.80',
  '80.80',
  '-',
  '0.60',
  '80.80',
  '3',
  '80.90',
  '2',
  '0.00'],
 ['0052',
  '富邦科技',
  '1,391,669',
  '1,974',
  '266,184,740',
  '189.90',
  '193.70',
  '189.45',
  '191.10',
  '-',
  '3.05',
  '191.05',
  '1',
  '191.10',
  '13',
  '0.00'],
 ['0053',
  '元大電子',
  '14,367',
  '110',
  '1,490,188',
  '103.50',
  '104.35',
  '103.00',
  '103.40',
  '-',
  '1.90',
  '103.15',
  '11',
  '103.50',
  '1',
  '0.00'],
 ['0055',
  '元大MSCI金融',
  '153,000',
  '345',
  '4,504,681',
  '29.50',
  '29.50',
  '29.36',
  '29.38',
  '-',
  '0.20',
  '29.38',
  '5',
  '29.39',
  '2',
  '0.00'],
 ['0056',
  '元大高股息',
  '102,173,261',
  '47,527',
  '3,973,836,071',
  '38.86',
  '39.04',
  '38.70',
  '39.02',
  '-',
  '0.08'

In [142]:
datas = pd.DataFrame(tds, columns=ths)
datas

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"18,604,770","22,549","3,594,464,975",192.45,195.30,191.65,193.00,-,3.40,193.00,113,193.05,41,0.00
1,0051,元大中型100,"115,782",353,"9,375,296",81.00,81.30,80.80,80.80,-,0.60,80.80,3,80.90,2,0.00
2,0052,富邦科技,"1,391,669","1,974","266,184,740",189.90,193.70,189.45,191.10,-,3.05,191.05,1,191.10,13,0.00
3,0053,元大電子,"14,367",110,"1,490,188",103.50,104.35,103.00,103.40,-,1.90,103.15,11,103.50,1,0.00
4,0055,元大MSCI金融,"153,000",345,"4,504,681",29.50,29.50,29.36,29.38,-,0.20,29.38,5,29.39,2,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,00953B,群益優選非投等債,"8,354,417",714,"84,697,218",10.13,10.15,10.13,10.14,+,0.02,10.13,"2,051",10.14,79,0.00
170,00954,中信日本半導體,"5,717,798",897,"51,690,002",9.10,9.12,9.00,9.08,-,0.42,9.07,297,9.08,48,0.00
171,00956,中信日經高股息,"851,304",275,"8,378,315",9.82,9.87,9.82,9.82,-,0.05,9.81,180,9.82,87,0.00
172,00960,野村全球航運龍頭,"11,416,954","5,373","169,681,770",14.95,14.95,14.78,14.83,-,0.17,14.83,295,14.84,81,0.00


In [143]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   證券代號     174 non-null    object
 1   證券名稱     174 non-null    object
 2   成交股數     174 non-null    object
 3   成交筆數     174 non-null    object
 4   成交金額     174 non-null    object
 5   開盤價      174 non-null    object
 6   最高價      174 non-null    object
 7   最低價      174 non-null    object
 8   收盤價      174 non-null    object
 9   漲跌(+/-)  174 non-null    object
 10  漲跌價差     174 non-null    object
 11  最後揭示買價   174 non-null    object
 12  最後揭示買量   174 non-null    object
 13  最後揭示賣價   174 non-null    object
 14  最後揭示賣量   174 non-null    object
 15  本益比      174 non-null    object
dtypes: object(16)
memory usage: 21.9+ KB


In [144]:
# 將新增欄位，並指定欄位位置 → 漲跌價 = 漲跌(+/-) + 漲跌價差
datas.insert(loc=9, column="漲跌價", value=datas["漲跌(+/-)"]+datas["漲跌價差"])

In [145]:
datas

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"18,604,770","22,549","3,594,464,975",192.45,195.30,191.65,193.00,-3.40,-,3.40,193.00,113,193.05,41,0.00
1,0051,元大中型100,"115,782",353,"9,375,296",81.00,81.30,80.80,80.80,-0.60,-,0.60,80.80,3,80.90,2,0.00
2,0052,富邦科技,"1,391,669","1,974","266,184,740",189.90,193.70,189.45,191.10,-3.05,-,3.05,191.05,1,191.10,13,0.00
3,0053,元大電子,"14,367",110,"1,490,188",103.50,104.35,103.00,103.40,-1.90,-,1.90,103.15,11,103.50,1,0.00
4,0055,元大MSCI金融,"153,000",345,"4,504,681",29.50,29.50,29.36,29.38,-0.20,-,0.20,29.38,5,29.39,2,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,00953B,群益優選非投等債,"8,354,417",714,"84,697,218",10.13,10.15,10.13,10.14,+0.02,+,0.02,10.13,"2,051",10.14,79,0.00
170,00954,中信日本半導體,"5,717,798",897,"51,690,002",9.10,9.12,9.00,9.08,-0.42,-,0.42,9.07,297,9.08,48,0.00
171,00956,中信日經高股息,"851,304",275,"8,378,315",9.82,9.87,9.82,9.82,-0.05,-,0.05,9.81,180,9.82,87,0.00
172,00960,野村全球航運龍頭,"11,416,954","5,373","169,681,770",14.95,14.95,14.78,14.83,-0.17,-,0.17,14.83,295,14.84,81,0.00


In [146]:
# 刪除 漲跌(+/-) 、 漲跌價差 這 2 個欄位。iloc的欄位置→10,11；inplace=True→改變原始資料(直接覆寫)
datas.drop(datas.iloc[:,[10, 11, 16]],axis=1,inplace=True)

In [147]:
datas

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量
0,0050,元大台灣50,"18,604,770","22,549","3,594,464,975",192.45,195.30,191.65,193.00,-3.40,193.00,113,193.05,41
1,0051,元大中型100,"115,782",353,"9,375,296",81.00,81.30,80.80,80.80,-0.60,80.80,3,80.90,2
2,0052,富邦科技,"1,391,669","1,974","266,184,740",189.90,193.70,189.45,191.10,-3.05,191.05,1,191.10,13
3,0053,元大電子,"14,367",110,"1,490,188",103.50,104.35,103.00,103.40,-1.90,103.15,11,103.50,1
4,0055,元大MSCI金融,"153,000",345,"4,504,681",29.50,29.50,29.36,29.38,-0.20,29.38,5,29.39,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,00953B,群益優選非投等債,"8,354,417",714,"84,697,218",10.13,10.15,10.13,10.14,+0.02,10.13,"2,051",10.14,79
170,00954,中信日本半導體,"5,717,798",897,"51,690,002",9.10,9.12,9.00,9.08,-0.42,9.07,297,9.08,48
171,00956,中信日經高股息,"851,304",275,"8,378,315",9.82,9.87,9.82,9.82,-0.05,9.81,180,9.82,87
172,00960,野村全球航運龍頭,"11,416,954","5,373","169,681,770",14.95,14.95,14.78,14.83,-0.17,14.83,295,14.84,81


In [148]:
datas["成交股數"] = datas["成交股數"].apply(lambda x:eval(x.replace(",","")))

In [149]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   證券代號    174 non-null    object
 1   證券名稱    174 non-null    object
 2   成交股數    174 non-null    int64 
 3   成交筆數    174 non-null    object
 4   成交金額    174 non-null    object
 5   開盤價     174 non-null    object
 6   最高價     174 non-null    object
 7   最低價     174 non-null    object
 8   收盤價     174 non-null    object
 9   漲跌價     174 non-null    object
 10  最後揭示買價  174 non-null    object
 11  最後揭示買量  174 non-null    object
 12  最後揭示賣價  174 non-null    object
 13  最後揭示賣量  174 non-null    object
dtypes: int64(1), object(13)
memory usage: 19.2+ KB


In [155]:
datas.sort_values("成交股數", ascending=False)[:20]

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量
22,00637L,元大滬深300正2,203028339,"26,398","3,302,811,098",16.15,16.60,16.06,16.30,-0.63,16.30,186,16.31,58
16,00632R,元大台灣50反1,189803293,"5,138","623,627,040",3.30,3.31,3.26,3.28,+0.04,3.28,"7,890",3.29,"27,538"
5,0056,元大高股息,102173261,"47,527","3,973,836,071",38.86,39.04,38.70,39.02,-0.08,39.02,235,39.03,35
97,00753L,中信中國50正2,98199477,"12,932","934,115,695",9.39,9.76,9.33,9.58,-0.36,9.58,873,9.59,16
60,00680L,元大美債20正2,93749340,"7,103","832,224,815",8.86,8.89,8.85,8.88,+0.19,8.88,480,8.89,"5,417"
48,00665L,富邦恒生國企正2,84773057,"7,777","726,536,915",8.39,8.77,8.35,8.69,-0.17,8.69,56,8.70,30
146,00922,國泰台灣領袖50,74652348,"22,126","1,683,008,977",22.64,22.73,22.33,22.62,-0.22,22.62,340,22.63,609
67,00688L,國泰20年美債正2,64245309,"3,747","566,969,516",8.81,8.84,8.80,8.84,+0.20,8.83,"1,008",8.84,"4,737"
143,00919,群益台灣精選高息,58821147,"29,081","1,386,422,235",23.49,23.67,23.38,23.59,-0.01,23.58,837,23.59,662
152,00929,復華台灣科技優息,56309179,"23,779","1,069,860,842",18.89,19.06,18.85,19.05,-0.01,19.05,497,19.06,"1,429"


In [156]:
datas.drop(datas.iloc[:, 10:14], axis=1, inplace=True)

In [157]:
columns = datas.columns
columns

Index(['證券代號', '證券名稱', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價',
       '漲跌價'],
      dtype='object')

In [158]:
columns[:10]

Index(['證券代號', '證券名稱', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價',
       '漲跌價'],
      dtype='object')

In [154]:
datas.values

array([['0050', '元大台灣50', 18604770, ..., '113', '193.05', '41'],
       ['0051', '元大中型100', 115782, ..., '3', '80.90', '2'],
       ['0052', '富邦科技', 1391669, ..., '1', '191.10', '13'],
       ...,
       ['00956', '中信日經高股息', 851304, ..., '180', '9.82', '87'],
       ['00960', '野村全球航運龍頭', 11416954, ..., '295', '14.84', '81'],
       ['00961', 'FT臺灣永續高息', 8889266, ..., '25', '9.99', '96']],
      dtype=object)